### Imports

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import subjects
from neuropy import plotting
from neuropy.utils import signal_process
from neuropy.analyses import Pf1D
from neuropy.analyses import Decode1d
from tqdm.notebook import tqdm
from neuropy.core import Epoch
from neuropy.utils.mathutil import min_max_scaler
import pandas as pd
from time import time
from numba import jit,njit


sessions = subjects.nsd.ratUday2


In [2]:
for sub, sess in enumerate(sessions):
    maze = sess.paradigm["maze"].flatten()
    post = sess.paradigm["post"].flatten()
    neurons = sess.neurons.get_neuron_type(neuron_type="pyr")
    pos = sess.maze
    # pos.t_start = pos.t_start - 0.5
    # run = sess.run
    pf = Pf1D(
        neurons=neurons,
        position=pos,
        speed_thresh=5,
        sigma=4,
        grid_bin=2,
        # epochs=run,
        frate_thresh=1,
    )
    pf_neurons = neurons.get_by_id(pf.neuron_ids)
    epochs = sess.pbe.time_slice(post[0]+500, post[0] + 1000)
    decode = Decode1d(
        neurons=pf_neurons,
        ratemap=pf,
        epochs=epochs,
        bin_size=0.02,
        decode_margin=15,
        nlines=8000,
    )

In [ ]:
@njit
def _decoder(spkcount, ratemaps):
    """
    ===========================
    Probability is calculated using this formula
    prob = ((frate)^nspike) * exp(-tau * frate)
    where,
        tau = binsize

    ===========================
    """
    tau = 0.02
    n_positions, n_time_bins = ratemaps.shape[1], spkcount.shape[1]

    prob = np.zeros((n_positions, n_time_bins))
    for i in range(n_positions):
        frate = (ratemaps[:, i, np.newaxis]) ** spkcount
        exp_frate = np.exp(-tau * np.sum(ratemaps[:, i]))
        prob[i, :] = np.prod(frate, axis=0) * exp_frate

    # old_settings = np.seterr(all="ignore")
    prob /= np.sum(prob, axis=0, keepdims=True)
    # np.seterr(**old_settings)

    return prob



In [ ]:
tic=time()
posterior = _decoder(np.hstack(decode.spkcount),pf.tuning_curves)
print(time()-tic)

In [ ]:
@jit(parallel=True)
def fit_post(arr,nlines,dt=1,dx=1):
    for i in range(10):
        t = np.arange(arr.shape[1])
        nt = len(t)
        tmid = (nt + 1) / 2 - 1

        pos = np.arange(arr.shape[0])
        npos = len(pos)
        pmid = (npos + 1) / 2 - 1

        phi = np.random.uniform(low=-np.pi / 2, high=np.pi / 2, size=nlines)
        diag_len = np.sqrt((nt - 1) ** 2 + (npos - 1) ** 2)
        rho = np.random.uniform(low=-diag_len / 2, high=diag_len / 2, size=nlines)
        # toc = time()
        # print(toc-tic)

        # tic = time()
        rho_mat = np.tile(rho, (nt, 1)).T
        phi_mat = np.tile(phi, (nt, 1)).T
        t_mat = np.tile(t, (nlines, 1))
        posterior = np.zeros((nlines, nt))
        # toc = time()
        # print(toc-tic)

        # tic=time()
        y_line = ((rho_mat - (t_mat - tmid) * np.cos(phi_mat)) / np.sin(phi_mat)) + pmid
        y_line = np.rint(y_line).astype("int")
        t_out = np.where((y_line < 0) | (y_line > npos - 1))
        t_in = np.where((y_line >= 0) & (y_line <= npos - 1))
        posterior[t_out] = np.median(arr[:, t_out[1]], axis=0)
        posterior[t_in] = arr[y_line[t_in], t_in[1]]

        posterior_mean = np.nanmean(posterior, axis=1)

        # tic=time()
        best_line = np.argmax(posterior_mean)
        score = posterior_mean[best_line]
        best_phi, best_rho = phi[best_line], rho[best_line]

        time_mid, pos_mid = nt * dt / 2, npos * dx / 2

        velocity = dx / (dt * np.tan(best_phi))
        intercept = (
            (dx * time_mid) / (dt * np.tan(best_phi))
            + (best_rho / np.sin(best_phi)) * dx
            + pos_mid
        )
        # return score,-velocity,intercept

def radon_transform(arr, nlines=10000, dt=1, dx=1, neighbours=1):


    # arr = np.apply_along_axis(
    #     np.convolve, axis=0, arr=arr, v=np.ones(2 * neighbours + 1), mode="same"
    # )
    return fit_post(arr,nlines,1,1)





In [ ]:
# from 
tic = time()
a = radon_transform(decode.posterior[1])
print(time()-tic)